In [2]:
import json
import os
from library_dicom.dicom_processor.tools.folders import *

from library_dicom.dicom_processor.tools.create_mip import *
from library_dicom.dicom_processor.tools.threshold_mask import *

import numpy as np
import SimpleITK as sitk

import csv

In [3]:
#PET4

csv_pet4_path = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_CSV_DATA/Export_Gained/data.csv'

with open(csv_pet4_path, 'r') as csv_file :
    reader = csv.reader(csv_file, delimiter = ',') #liste pour chaque ligne 
    csv_data = []
    for row in reader :
        csv_data.append(row)
        
del csv_data[0] #enlever première ligne


In [4]:
csv_data[1]

['alina',
 '11011101021002',
 '0',
 '18',
 '1.2.250.1.59.470.940100027.1.20130128113103.7763.17688',
 '2013-01-30',
 '0',
 '0',
 '0',
 'review18.csv',
 '4',
 '29.35',
 'Abdominal',
 '13.6',
 'Abdominal',
 '1.35',
 '1.17',
 '0.1',
 '1.84',
 '1.49',
 '0.11',
 '1.99',
 '1.52',
 '0.17',
 '0',
 '12',
 '3.17',
 '2.36',
 '0.49',
 '2',
 '2',
 '19',
 '1.2.840.113704.1.111.2180.1361963756.13',
 '2013-02-27',
 '0',
 '0',
 '0',
 'review19.csv',
 '1.96',
 'Intestine - small and colon',
 '1.53',
 '1.4',
 '0.12',
 '2.27',
 '1.86',
 '0.16',
 '1',
 '0',
 '0',
 '4',
 '20',
 '1.2.840.113704.1.111.4636.1364288605.3',
 '2013-03-26',
 '0',
 '0',
 '0',
 'review20.csv',
 '1.96',
 'Intestine - small and colon',
 '1.53',
 '1.4',
 '0.12',
 '2.27',
 '1.86',
 '0.16',
 '1',
 '0',
 '0']

In [5]:
nifti_directory = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI_PET4'
liste_nifti = os.listdir(nifti_directory)

In [6]:
liste = []
for row in csv_data : 
    subliste = []
    #subliste.append(row[0])
    subliste.append(row[1])
    subliste.append(row[51])
    study_uid = row[51]
    subsubliste = []
    for nifti in liste_nifti : 
        if study_uid in nifti: 
            subsubliste.append(nifti)
    if len(subsubliste) == 2 : 
        subsubliste = sorted(subsubliste)
        #print(subsubliste)
        subliste.append(subsubliste[0])
        subliste.append(subsubliste[1])
        #subliste.append(subsubliste[2])
    else : 
        pass
    subliste.append("pet4")
    liste.append(subliste)



In [7]:
len(liste)

728

In [8]:
liste[10]

['11011101021011',
 '1.2.250.1.59.470.940100027.1.20140619143155.5902.130185',
 '1.2.250.1.59.470.940100027.1.20140619143155.5902.130185_nifti_CT.nii',
 '1.2.250.1.59.470.940100027.1.20140619143155.5902.130185_nifti_PT.nii',
 'pet4']

In [9]:
#clean -> if len != 6 
#doit y avoir 573
completed_liste = []
for serie in liste : 
    if len(serie) == 5 : 
        completed_liste.append(serie)


In [10]:
len(completed_liste)

586

In [11]:
new_liste = []
new_liste.append(completed_liste[0])
for i in range(1, len(completed_liste)) :
    if completed_liste[i] not in new_liste : 
        new_liste.append(completed_liste[i])

In [12]:
len(new_liste)

527

In [24]:
new_liste[10]

['11011101021014',
 '1.2.840.113704.1.111.3608.1434978016.23',
 '1.2.840.113704.1.111.3608.1434978016.23_nifti_CT.nii',
 '1.2.840.113704.1.111.3608.1434978016.23_nifti_PT.nii',
 'pet2']

In [13]:
import shutil

In [16]:
new_liste[1]

['11011101021004',
 '1.2.840.113704.1.111.2304.1369728102.4',
 '1.2.840.113704.1.111.2304.1369728102.4_nifti_CT.nii',
 '1.2.840.113704.1.111.2304.1369728102.4_nifti_PT.nii',
 'pet4']

In [19]:
#creation sous dossier patient id -> pet0 -> mask ct pt 

for serie in new_liste: 
    patient_id = serie[0]
    pet_type = serie[-1]
    nifti_ct = serie[2]
    nifti_pet = serie[3]
        #nifti_mask = serie[5]

    folder = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI'+'/'+patient_id

    if not os.path.exists(folder):
        os.makedirs(folder)
        print(folder, 'created')
    else : print(folder, "Dossier patient déjà crée")

    subfolder = folder+'/'+pet_type
    if not os.path.exists(subfolder) :
        os.makedirs(subfolder)
        print(subfolder, 'created')
    else : print(subfolder, "déjà crée => prblm")

    ct_path = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI_PET4'+'/'+nifti_ct
    pt_path = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI_PET4'+'/'+nifti_pet
        #mask_path = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI'+'/'+nifti_mask

    new_ct_path = subfolder+'/'+nifti_ct
    new_pet_path = subfolder+'/'+nifti_pet
        #new_mask_path = subfolder+'/'+nifti_mask
    try : 

        os.rename(ct_path, subfolder+'/'+nifti_ct)
        os.rename(pt_path, subfolder+'/'+nifti_pet)
        #os.rename(mask_path, subfolder+'/'+nifti_mask)
    except Exception as err : 
        print(err)




    



8024b95f92/GAINED_NIFTI/11011108371001 Dossier patient déjà crée
/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI/11011108371001/pet4 created
/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI/11011108371002 Dossier patient déjà crée
/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI/11011108371002/pet4 created
/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI/11011108371003 Dossier patient déjà crée
/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI/11011108371003/pet4 created
/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI/11011108371004 Dossier patient déjà crée
/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI/11011108371004/pet4 created
/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI/11011108611001 Dossier patient déjà crée
/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI/11011108611001/pet4 created
/

In [ ]:
#REUNITE CSV 
